## 1.导入数据

In [5]:
import pandas as pd
data=pd.read_table('data/bug_data.txt',sep='\t',header=None,nrows = 10000,names=["标签","短信内容"])
data.head()

,标签,短信内容
0,0,开了加速依然显示服务器连接失败
1,1,正常使用中会闪退
2,1,开机启动任务计划请求的操作需要提升
3,0,windows睡眠 遊戲時數
4,1,GITHUB加速会出现SSL连不上，GITHUB.com 443报警


In [6]:
data.shape

(90, 2)

## 2.进行分词

In [7]:
import jieba
data['分词后数据']=data["短信内容"].apply(lambda x:' '.join(jieba.cut(x)))
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hp\AppData\Local\Temp\jieba.cache
Loading model cost 0.630 seconds.
Prefix dict has been built successfully.


,标签,短信内容,分词后数据
0,0,开了加速依然显示服务器连接失败,开 了 加速 依然 显示 服务器 连接 失败
1,1,正常使用中会闪退,正常 使用 中会 闪退
2,1,开机启动任务计划请求的操作需要提升,开机 启动 任务 计划 请求 的 操作 需要 提升
3,0,windows睡眠 遊戲時數,windows 睡眠 遊戲 時數
4,1,GITHUB加速会出现SSL连不上，GITHUB.com 443报警,GITHUB 加速 会 出现 SSL 连不上 ， GITHUB . com 443 报警


## 3.导入停用词

In [9]:
f = open('data/my_stop_words.txt','r')
my_stop_words_data = f.readlines()
f.close()
my_stop_words_list=[]
for each in my_stop_words_data:
    my_stop_words_list.append(each.strip('\n'))

## 4.提取特征和目标数据

In [10]:
X = data['分词后数据']
y = data['标签']

## 5.模型训练and预测打分

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

skf = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

for train_index, test_index in skf.split(X, y):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    pipeline = Pipeline([
                        ('vect', TfidfVectorizer(stop_words=my_stop_words_list)),
                        ('clf', MultinomialNB(alpha=1.0))])
    
    pipeline.fit(X_train,y_train)
    
    #进行预测
    predict = pipeline.predict(X_test)
    score = pipeline.score(X_test,y_test) 
    print(score)  

0.6666666666666666
0.4444444444444444
0.7777777777777778
0.4444444444444444
0.5555555555555556
0.4444444444444444
0.5555555555555556
0.5555555555555556
0.7777777777777778
0.4444444444444444


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  warnings.warn(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  warnings.warn(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  warnings.warn(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokeni

In [13]:
data["数据类型"] = pipeline.predict(X) #lambda x:x+1 if not 2==1 else 0  
data['数据类型']=data["数据类型"].apply(lambda x:"bug" if x==1 else "其他")
data.head()

,标签,短信内容,分词后数据,数据类型
0,0,开了加速依然显示服务器连接失败,开 了 加速 依然 显示 服务器 连接 失败,其他
1,1,正常使用中会闪退,正常 使用 中会 闪退,bug
2,1,开机启动任务计划请求的操作需要提升,开机 启动 任务 计划 请求 的 操作 需要 提升,bug
3,0,windows睡眠 遊戲時數,windows 睡眠 遊戲 時數,其他
4,1,GITHUB加速会出现SSL连不上，GITHUB.com 443报警,GITHUB 加速 会 出现 SSL 连不上 ， GITHUB . com 443 报警,bug
